In [0]:
from pyspark.sql.functions import current_timestamp

def load_to_bronze_table(parquet_path: str, table_name: str):

    table_exists = spark.catalog.tableExists(table_name)
    df = spark.read.parquet(parquet_path)
    df = df.withColumn('inserted_at', current_timestamp())
    if table_exists:
        df.write.format("delta").mode("append").saveAsTable(table_name)
    else:
        df.write.format("delta").mode("overwrite").saveAsTable(table_name)


def cleanup(table_name):
    
    table_path = f"/user/hive/warehouse/{table_name}"
    try:
        dbutils.fs.rm(f"dbfs:{table_path}", True)
    except:
        print(f"{table} files Not found")
    try:
        spark.sql(f"DROP TABLE {table}")
    except:
        print(f"{table} Not found")

In [0]:
file_table_mapping = {
    "/FileStore/shared_uploads/saiprasadpadhy@gmail.com/media_customer_reviews.parquet": "media_customer_reviews_bronze",
    "/FileStore/shared_uploads/saiprasadpadhy@gmail.com/media_gold_reviews_chunked.parquet": "media_gold_reviews_chunked_bronze",
    "/FileStore/shared_uploads/saiprasadpadhy@gmail.com/sales_customers.parquet": "sales_customers_bronze",
    "/FileStore/shared_uploads/saiprasadpadhy@gmail.com/sales_franchises.parquet": "sales_franchises_bronze",
    "/FileStore/shared_uploads/saiprasadpadhy@gmail.com/sales_suppliers.parquet": "sales_suppliers_bronze",
    "/FileStore/shared_uploads/saiprasadpadhy@gmail.com/sales_transactions.parquet": "sales_transactions_bronze",
}

for path, table in file_table_mapping.items():
    load_to_bronze_table(path, table)
    #cleanup(table)